In [ ]:
from aif360.algorithms.postprocessing import EqOddsPostprocessing
from sklearn.metrics import accuracy_score

# Initialisation de l'algorithme Equalized Odds Postprocessing
eq_odds = EqOddsPostprocessing(
    unprivileged_groups=[{protected_attribute: 0}],
    privileged_groups=[{protected_attribute: 1}],
    seed=42
)

# Ajustement des prédictions sur le dataset de validation
eq_odds.fit(aif_df_valid, aif_df_valid)

# Transformation des prédictions
transformed_eq_odds = eq_odds.predict(aif_df_valid)

# Calcul des métriques après post-traitement
print("Metrics after Equalized Odds Postprocessing:")
print_metrics(
    labels=aif_df_valid.labels.ravel(),
    preds=transformed_eq_odds.labels.ravel(),
    gender_or_age=1  # 0 pour 'Patient Gender', 1 pour 'age_30_65'
)

# Évaluation des performances
accuracy = accuracy_score(aif_df_valid.labels, transformed_eq_odds.labels)
print(f"Accuracy after Equalized Odds Postprocessing: {accuracy:.4f}")

In [ ]:
from aif360.algorithms.postprocessing import RejectOptionClassification

# Initialisation de l'algorithme Reject Option Classification
roc = RejectOptionClassification(
    unprivileged_groups=[{protected_attribute: 0}],
    privileged_groups=[{protected_attribute: 1}],
    low_class_thresh=0.01,
    high_class_thresh=0.99,
    num_class_thresh=100,
    num_ROC_margin=50,
    metric_name="Statistical parity difference",
    metric_ub=0.05,
    metric_lb=-0.05,
)

# Ajustement des prédictions sur le dataset de validation
roc.fit(aif_df_valid, aif_df_valid)

# Transformation des prédictions
transformed_roc = roc.predict(aif_df_valid)

# Calcul des métriques après post-traitement
print("Metrics after Reject Option Classification:")
print_metrics(
    labels=aif_df_valid.labels.ravel(),
    preds=transformed_roc.labels.ravel(),
    gender_or_age=1  # 0 pour 'Patient Gender', 1 pour 'age_30_65'
)

# Évaluation des performances
accuracy = accuracy_score(aif_df_valid.labels, transformed_roc.labels)
print(f"Accuracy after Reject Option Classification: {accuracy:.4f}")

In [ ]:
# Matrice de confusion globale après Equalized Odds
plot_confusion_matrix(
    y_true=aif_df_valid.labels.ravel(),
    y_pred=transformed_eq_odds.labels.ravel(),
    normalize=True,
    title="Matrice de confusion après Equalized Odds"
)

# Matrice de confusion globale après Reject Option Classification
plot_confusion_matrix(
    y_true=aif_df_valid.labels.ravel(),
    y_pred=transformed_roc.labels.ravel(),
    normalize=True,
    title="Matrice de confusion après Reject Option Classification"
)

# Matrice de confusion par groupe (par exemple, age_30_65) après Equalized Odds
plot_confusion_matrix_by_group(
    y_true=aif_df_valid.labels.ravel(),
    y_pred=transformed_eq_odds.labels.ravel(),
    df=df_encoded_bool,
    group_columns=["age_30_65"],
    labels=["sain", "malade"],
    normalize=True
)

# Matrice de confusion par groupe (par exemple, age_30_65) après Reject Option Classification
plot_confusion_matrix_by_group(
    y_true=aif_df_valid.labels.ravel(),
    y_pred=transformed_roc.labels.ravel(),
    df=df_encoded_bool,
    group_columns=["age_30_65"],
    labels=["sain", "malade"],
    normalize=True
)

In [ ]:
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing

# Initialisation
calibrated_eq_odds = CalibratedEqOddsPostprocessing(
    unprivileged_groups=[{protected_attribute: 0}],
    privileged_groups=[{protected_attribute: 1}],
    cost_constraint="fnr",  # ou "fpr", ou "weighted"
    seed=42
)

# Ajustement des prédictions
calibrated_eq_odds.fit(aif_df_valid, aif_df_valid)

# Transformation des prédictions
transformed_calibrated_eq_odds = calibrated_eq_odds.predict(aif_df_valid)

# Calcul des métriques
print("Metrics after Calibrated Equalized Odds Postprocessing:")
print_metrics(
    labels=aif_df_valid.labels.ravel(),
    preds=transformed_calibrated_eq_odds.labels.ravel(),
    gender_or_age=1
)

In [ ]:
from aif360.algorithms.postprocessing import ThresholdOptimizer

# Initialisation
threshold_optimizer = ThresholdOptimizer(
    unprivileged_groups=[{protected_attribute: 0}],
    privileged_groups=[{protected_attribute: 1}],
    estimator=None,  # Peut être un modèle pré-entraîné
    constraints="equalized_odds",  # ou "demographic_parity"
)

# Ajustement des prédictions
threshold_optimizer.fit(aif_df_valid)

# Transformation des prédictions
transformed_threshold_optimizer = threshold_optimizer.predict(aif_df_valid)

# Calcul des métriques
print("Metrics after Threshold Optimizer Postprocessing:")
print_metrics(
    labels=aif_df_valid.labels.ravel(),
    preds=transformed_threshold_optimizer.labels.ravel(),
    gender_or_age=1
)